In [5]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition, pipeline, metrics
from sklearn.model_selection import GridSearchCV



In [7]:
air_visit_data = pd.read_csv('air_visit_data.csv.zip')
sample_submission = pd.read_csv('sample_submission.csv.zip')
sample_submission['air_store_id'] = sample_submission['id'].apply(lambda x: x.split('_')[0]+str('_') +x.split('_')[1])
sample_submission['visit_date'] = sample_submission['id'].apply(lambda x: x.split('_')[2])
sample_submission['visit_date'] = pd.to_datetime(sample_submission['visit_date'],format= '%Y-%m-%d %H:%M:%S')

In [8]:
air_visit_data['visit_date'] = pd.to_datetime(air_visit_data['visit_date'],format= '%Y-%m-%d')
air_visit_data.visit_date.min()

Timestamp('2016-01-01 00:00:00')

In [9]:
air_visit_data['visit_date_month'] =air_visit_data.visit_date.dt.month
air_visit_data['visit_date_dayofw'] =air_visit_data.visit_date.dt.dayofweek
air_visit_data['visit_date_year'] =air_visit_data.visit_date.dt.year
air_visit_data['visit_date_dayofm'] =air_visit_data.visit_date.dt.day
air_visit_data['weekofyear'] =air_visit_data.visit_date.dt.weekofyear

In [10]:
df=air_visit_data.groupby(['air_store_id','visit_date_month']).agg('mean')['visitors'].reset_index()

In [11]:
df.columns

Index(['air_store_id', 'visit_date_month', 'visitors'], dtype='object')

In [12]:
df = df.pivot(index='air_store_id', columns='visit_date_month', values='visitors').reset_index()

In [13]:
df = df[['air_store_id', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]]

In [14]:
df.columns = ['air_store_id', 'm0', 'm1', 'm2', 'm3', 'm4', 'm5', 'm6','m7', 'm8', 'm9', 'm10', 'm11']

In [15]:
df.isnull().sum()

air_store_id      0
m0               12
m1               10
m2                1
m3                4
m4              515
m5              515
m6               30
m7               27
m8               22
m9               20
m10              18
m11              18
dtype: int64

In [16]:
df = df.drop(['m4','m5'],axis=1)

In [17]:
df.replace(np.nan,0,inplace = True)

In [18]:
df.to_csv('air_store_month_wise_mean.csv',index= False)

In [19]:
df

,air_store_id,m0,m1,m2,m3,m6,m7,m8,m9,m10,m11
0,air_00a91d42b08b08d9,25.869565,29.652174,29.384615,27.222222,26.080000,24.166667,22.833333,23.250000,22.625000,30.428571
1,air_0164b9927d20bcc3,10.789474,7.476190,9.625000,8.315789,0.000000,0.000000,0.000000,10.000000,8.260870,10.333333
2,air_0241aa3964b7f861,9.540000,9.212766,9.962264,10.250000,8.178571,10.125000,10.954545,9.153846,9.600000,12.000000
3,air_0328696196e46f18,7.714286,7.571429,9.470588,8.384615,8.000000,6.100000,10.500000,10.666667,3.888889,10.666667
4,air_034a3d5b40d5b1b1,5.733333,27.428571,23.806452,22.409091,13.777778,11.444444,9.333333,7.407407,12.272727,9.357143
...,...,...,...,...,...,...,...,...,...,...,...
824,air_fea5dc9594450608,13.807692,18.041667,15.866667,16.227273,13.586207,13.750000,13.423077,12.433333,11.333333,17.344828
825,air_fee8dcf4d619598e,22.633333,27.714286,24.903226,26.727273,29.193548,26.233333,27.666667,23.838710,22.633333,29.640000
826,air_fef9ccb3ba0da2f7,6.565217,6.041667,7.000000,9.421053,13.296296,10.772727,12.384615,10.333333,11.120000,8.720000
827,air_ffcc2d5087e1b476,22.913043,19.791667,20.857143,19.500000,20.884615,18.090909,19.200000,18.560000,21.440000,20.880000
